## Данные нормированы:


In [1]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import Model
from keras.layers import Dense, Input, Conv2D
from keras.callbacks import ModelCheckpoint

from mne.io import read_raw_brainvision as rd
from os.path import join
from os import listdir
import numpy as np

Using TensorFlow backend.


In [3]:
def init_weights(shape, name=None):
    return initializers.normal(shape, scale=0.01, name=name)

channels_num = 61
window_size = 1
nb_epoch = 5

# Hyper parameters
batch_size = 64
nb_visible = channels_num * window_size
nb_emb = 15

corruption_level = 0.001

In [4]:
var, mean = None, None
with open('scaler_(var_mean).csv', 'r') as f:
    var, mean = f.read().split()
    var, mean = np.float(var), np.float(mean)

In [5]:
mne_format_data_dir = join('data', 'resting_state_mne_format')

In [6]:
# Build autoencoder model
encoder = Sequential()
encoder.add(Dense(2000, activation='sigmoid', input_shape=(nb_visible,) ))
encoder.add(Dense(1000, activation='sigmoid'))
encoder.add(Dense(500, activation='sigmoid'))
encoder.add(Dense(nb_emb))

decoder = Sequential()
decoder.add(Dense(2000, activation='sigmoid', input_shape=(nb_emb,) ))
decoder.add(Dense(1000, activation='sigmoid'))
decoder.add(Dense(500, activation='sigmoid'))
decoder.add(Dense(nb_visible))

input_img = Input(shape=(nb_visible,))
encoded = encoder(input_img)
decoded = decoder(encoded)

autoencoder = Model(inputs=input_img, outputs=decoded)
autoencoder.compile(optimizer='adadelta', loss='mse')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 61)                0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 15)                2633015   
_________________________________________________________________
sequential_2 (Sequential)    (None, 61)                2564061   
Total params: 5,197,076
Trainable params: 5,197,076
Non-trainable params: 0
_________________________________________________________________


In [7]:
def read_data(data_type, return_batch_np=False):
    verbose = None if return_batch_np else 0
    data = [rd(join(mne_format_data_dir, data_type, fn), verbose=verbose ).get_data().reshape(-1, nb_visible) \
            for fn in listdir(join(mne_format_data_dir, data_type)) \
            if fn[-5:] == '.vhdr']
    if return_batch_np:
        sz = sum([el.shape[0] for el in data])
        return sz // batch_size + (sz % batch_size != 0)
    data = np.vstack(np.array(data))
    data = (data - mean) / (var**(1/2))
    # make last batch
    if data.shape[0] % batch_size:
        data = np.vstack((data, data[:batch_size - data.shape[0] % batch_size, :]))
    return data.reshape(-1, batch_size, nb_visible)

def gen(data_type):
    data = read_data(data_type)
    for ep in range(nb_epoch+1):
        i = np.arange(data.shape[0])
        np.random.shuffle(i)
        data = data[i, :]
        for i in range(data.shape[0]):
            yield data[i], data[i]

In [ ]:
train_batches_nb = read_data('train', return_batch_np=True)
val_batches_nb = read_data('validation', return_batch_np=True)

Extracting parameters from data/resting_state_mne_format/train/2505_shirokova_post_eeg.vhdr...
Setting channel info structure...
Extracting parameters from data/resting_state_mne_format/train/miloslavov_22_05_pre_eeg.vhdr...
Setting channel info structure...
Extracting parameters from data/resting_state_mne_format/train/gorin_rest_eeg_post_31011200.vhdr...
Setting channel info structure...
Extracting parameters from data/resting_state_mne_format/train/2704_zagirova_post_eeg.vhdr...
Setting channel info structure...
Extracting parameters from data/resting_state_mne_format/train/2103_glebko_posteeg.vhdr...
Setting channel info structure...
Extracting parameters from data/resting_state_mne_format/train/200317_ivanova_pre_eeg.vhdr...
Setting channel info structure...
Extracting parameters from data/resting_state_mne_format/train/gorbacheva_03_02_2017_post_eeg.vhdr...
Setting channel info structure...
Extracting parameters from data/resting_state_mne_format/train/2403_kutuzova_pre_eeg.vhdr.

In [ ]:
history = autoencoder.fit_generator(gen('train'), steps_per_epoch=train_batches_nb, epochs=nb_epoch, \
                          validation_data=gen('validation'), validation_steps=val_batches_nb, \
                          use_multiprocessing=True, \
                          callbacks=[ModelCheckpoint("models/dense15_{epoch:02d}.hdf5", save_best_only=True)])

Epoch 1/5
208251/233528 [=========================>....] - ETA: 359s - loss: 1.1992

In [ ]:
history.history

In [9]:
import matplotlib.pyplot as plt
from keras.models import load_model

In [13]:
autoencoder = load_model('models/dense15_01.hdf5')

In [14]:
def metric(y_true, y_pred):
    l2ratio = np.sum((y_true - y_pred)**2) / np.sum(y_true**2)
    return np.sqrt(l2ratio) * 100

In [15]:
x = []
i = 0
for test in gen('test'):
    i += 1
    x.append(metric(autoencoder.predict(test[0]), test[1]))
    if (i % 25) == 0:
        print(np.mean(x))

2.89260755762
2.67284443853
2.55398186778
3.91165758977
4.13953291332
4.2241846239
4.75899212154
4.44678819615
4.33539417656
4.47137624275
4.39862397702
4.45739269653
4.49143811155
4.3940766719
4.37880014667
4.49651956041
4.70920280154
4.62324961982
4.50381654448
4.45887407566
4.34815562743
4.33950070789
4.29045960096
4.99007355769
4.96161063346
4.85335836194
4.88223792262
5.28967291126
5.25994592162
5.18642462512
5.15241101971
5.05328158235
4.99101050903
4.99521746979
4.92567969563
4.93915674155
4.90065815504
5.13719469212
5.08000456657
5.01008481474
4.94929925419
4.92619509755
5.29928881148
5.32729156113
5.33192390892
5.26720955211
5.57841606567


KeyboardInterrupt: 